# Autonomous driving - Car detection

The ideas in this notebook are described in the two YOLO papers: YOLOv3: An Incremental Improvement ( https://arxiv.org/abs/1804.02767 ). 

In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from yad2k.utils.yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, letterbox_image, draw_boxes, scale_boxes
from yad2k.keras_yolo import yolo_boxes_and_scores, yolo_head

%matplotlib inline

C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Introduction
Bounding box exmple :

<img src="notebook_images/rectangle.png" style="width:70%;height:70%;">


In this prject, I use pretrained weights, where we have 80 trained yolo classes, for recognition.  
The class label is represented as $c$ and it's integer from 1 to 80, each number represent the class label accordingly.  
If $c=3$, then the classified object is a $car$


## 2 - YOLO

YOLO stands for You Only Look Once. It's an object detector that uses features learned by a deep convolutional neural network to detect an object. Before we get out hands dirty with code, we must understand how YOLO works.

### 2.1 - Architercutre details

Neural network io:
-  **input** : (m, 416, 416, 3)
-  **output** : confidece of an object being present in the rectangle, list of rectangles position and sizes and classes of the objects begin detected. Each bounding box is represented by 6 numbers $(Pc, Rx, Ry, Rh, Rw, C1..Cn)$ as explained above. In this case n=80, which means we have $c$ as 80-dimensional vector, and the final size of representing the bounding box is 85 

The first detection is made by the 82nd layer. For the first 81 layers, the image is down sampled by the network, such that the 81st layer has a stride of 32. If we have an image of 416 x 416, the resultant feature map would be of size 13 x 13. One detection is made here using the 1 x 1 detection kernel, giving us a detection feature map of 13 x 13 x 3 x 85.

Then, the feature map from layer 79 is subjected to a few convolutional layers before being up sampled by 2x to dimensions of 26 x 26. This feature map is then depth concatenated with the feature map from layer 61. Then the combined feature maps is again subjected a few 1 x 1 convolutional layers to fuse the features from the earlier layer (61). Then, the second detection is made by the 94th layer, yielding a detection feature map of 26 x 26 x 3 x 85.

A similar procedure is followed again, where the feature map from layer 91 is subjected to few convolutional layers before being depth concatenated with a feature map from layer 36. Like before, a few 1 x 1 convolutional layers follow to fuse the information from the previous layer (36). We make the final of the 3 at 106th layer, yielding feature map of size 52 x 52 x 3 x 85.

YOLO v3, in total uses 9 anchor boxes. Three for each scale. 

<img src="notebook_images/architecture.png" style="width:95%;height:95%;">

<caption> Note : The last three terms of the loss function in YOLO v2 are the squared errors, whereas in YOLO v3, they’ve been replaced by cross-entropy error terms. In other words, object confidence and class predictions in YOLO v3 are now predicted through logistic regression.  
YOLO v3, in total uses 9 anchor boxes. Three for each scale. I</caption>

For keeping the things simple we will flatten the last two dimensions, from (19, 19, 5, 85) the output of our CNN is flatten to (19, 19, 425).


Now, for each box (of each cell) we will compute the following elementwise product and extract a probability that the box contains a certain class.

<img src="notebook_images/probability_extraction.png" style="width:700px;height:400;">
<caption><center>**Find the class detected by each box**<br> </center></caption>

Here's one example visualizing what YOLO can predict on an image:
- For each of the $SxS$ grid cells, find the maximum of the confidence scores (taking a max across both the 5 anchor boxes and across different classes).  
A confidence score is: probability(containing an object) x IoU(pred, truth).
If the cell contains an object, it predicts a probability of this object belonging to one class Ci, i=1,2,…, K: probability(the object belongs to the class Ci | containing an object). At this stage, the model only predicts one set of class probabilities per cell, regardless of the number of boxes B. 
- Color that grid cell according to what object that grid cell considers the most likely.
- Draw rectangle 

<img src="notebook_images/yolo.png" style="width:100%;height:100%;">
<caption> Each of the $SxS$ grid cells colored according the class that has the largest predicted probability in specified cell.</caption>



### Part 1 - Example Filtering with a threshold on class scores

The output result may contain several rectangles that are false positives or overlap, so we need to find a way to reduce them.
The first attempt to reduce these rectangles is to filter them by threshold.


**Input arguments**: 
- `box_confidence`: tensor of shape $(19 \times 19, 5, 1)$ containing $p_c$ (confidence probability that there's some object) for each of the 5 boxes predicted in each of the 19x19 cells.
- `boxes`: tensor of shape $(19 \times 19, 5, 4)$ containing $(b_x, b_y, b_h, b_w)$ for each of the 5 boxes per cell.
- `box_class_probs`: tensor of shape $(19 \times 19, 5, 80)$ containing the detection probabilities $(c_1, c_2, ... c_{80})$ for each of the 80 classes for each of the 5 boxes per cell.



In [2]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    
    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes
    
    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    
    # Step 1: Compute box scores
    box_scores = box_confidence * box_class_probs #multiply box probability(p) with class probability
    
    # Step 2: Find the box_classes thanks to the max box_scores, keep track of the corresponding score
    box_classes = K.argmax(box_scores, axis=-1) #max class index
    box_class_scores = K.max(box_scores, axis=-1) #max class probability
    
    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    filtering_mask = box_class_scores>=threshold
    
    # Step 4: Apply the mask to scores, boxes and classes
    scores = tf.boolean_mask(tensor=box_class_scores, mask=filtering_mask, name='score_mask')
    boxes  = tf.boolean_mask(tensor=boxes, mask=filtering_mask, name='box_mask')
    classes = tf.boolean_mask(tensor=box_classes, mask=filtering_mask, name='classes_mask')
    
    return scores, boxes, classes

### Part 2 : Non-max suppression ###

Even after yolo filtering by thresholding over, we still have a lot of overlapping boxes. Second approach and filtering is Non-Max suppression algorithm.

* Discard all boxes with $Pc <= 0.6$  
* While tehre are any remaining boxes : 
    * Pick the box with the largest $Pc$
    * Output that as a prediction
    * Discard any remaining boxes with $IOU>=0.5$ with the box output in the previous step

<img src="notebook_images/nms_algo.jpg" style="width:25%;height:25%;">
<caption>Example of non max suppression algorithm, on input the aglorithm receive 4 overlapping bounding boxes, and the output returns only one</caption>


Non-max suppression uses the very important function called **"Intersection over Union"**, or IoU.
<img src="notebook_images/iou.png" style="width:500px;height:400;">

**Below is implementation of iou**:

In [3]:

def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (x1, y1, x2, y2)
    box2 -- second box, list object with coordinates (x1, y1, x2, y2)
    """

    # Calculate the (y1, x1, y2, x2) coordinates of the intersection of box1 and box2. Calculate its Area.
    xi1 = max(box1[0],box2[0])
    yi1 = max(box1[1],box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = abs(xi1-xi2) * abs(yi1-yi2)

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = abs(box1[0]-box1[2]) * abs(box1[1]-box1[3])
    box2_area = abs(box2[0]-box2[2]) * abs(box2[1]-box2[3])
    union_area = box1_area + box2_area -inter_area
    
    # compute the IoU
    iou = inter_area/union_area

    return iou

In [4]:
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou = " + str(iou(box1, box2)))

iou = 0.14285714285714285


With computing iou, we co mposed the third part of non max supression algorithm, now, lets combine everything into one :

### 2.4 Wrapping up the filtering

Let's calculate the whole algorithm : 
* Get Yolo CNN output
* Discard all boxes with $Pc <= 0.6$  
* While tehre are any remaining boxes : 
    * Pick the box with the largest $Pc$
    * Output that as a prediction
    * Discard any remaining boxes with $IOU>=0.5$ with the box output in the previous step

In [5]:

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
    """
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    nms_indices = tf.image.non_max_suppression(boxes=boxes, 
                                               scores=scores,
                                               max_output_size=max_boxes, 
                                               iou_threshold=iou_threshold, name='nms_indices')
    
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [6]:

def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    # 1. Evaluate YOLO model on given input and return filtered boxes.
    # 2. Scale boxes back to original image shape.
    # 3. Filter boxes
    # 4. Apply non max suppression algorithm
    
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores (yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)


    return boxes_, scores_, classes_

## Part 3 :  YOLO V2. Model

For the purpose of this project, I'm using a pretrained weights for Yolo V2.
You can download the weights <a href = "http://pjreddie.com/media/files/yolo.weights"> here</a>

In [7]:
sess = K.get_session()

### 3.1 - Defining classes, anchors and image shape.

We have gathered the information about the 80 classes and 5 boxes in two files "coco_classes.txt" and "yolo_anchors.txt". Let's load these quantities into the model by running the next cell. 

The car detection dataset has 720x1280 images, which we've pre-processed into 608x608 images. 

# Anchor Boxes

Generating anchor boxes using K-means clustering

There are many ways to compute bounding boxes for detection tasks. One approach is to directly predict the bounding box values, however this approach is susceptible to errors as it tends to favor bounding boxes with large dimensions. Further, the training process is unstable because the range of values to predict can vary significantly.In YOLOv2, the first step is to compute good candidate anchor boxes. This is achieved using K-means clustering. However, using direct Euler distance metric for K-means minizers error for larger bounding boxes, but not for smaller boxes. Therefore, in YOLOv2, intersection over union (IOU) is used as a distance metric. The IOU calculations are made assuming all the bounding boxes are located at one point, i.e. only width and height are used as features. Figure below shows the height and width plotted against each other. Fixed slopes indicate that most bounding boxes have specific predefined aspect ratios, and size. This is not surprising given the fact that a person and vehicle are expected to have certain fixed dimensions.


<img src="notebook_images/clustering.png" style="width:500px;height:400;">


In Yolo V2, this specialization is ‘assisted’ with predefined anchors as in Faster-RCNN. The predefined anchors are chosen to be as representative as possible of the ground truth boxes, with the following K-means clustering algorithm to define them:

* all ground-truth bounding boxes are centered on (0,0)

* the algorithm initiates 5 centroïds by drawing randomly 5 of the ground-truth bounding boxes

* then, the following two steps are alternated:

    * each ground truth box is assigned to one of the centroïd, using as distance measure the IOU, in order to get 5 clusters or groups of ground-truth bouding boxes

    * new centroids are computed by taking the box inside each cluster that minimizes the mean IOU with all other boxes inside the cluster


# Reading class/anchors data
Bellow we are reading the class labels and pre-computed anchors points with K-means algorithm

To determine the priors, YOLOv3 applies k-means cluster. Then it pre-select 9 clusters. For COCO, the width and height of the anchors are (10×13),(16×30),(33×23),(30×61),(62×45),(59× 119),(116 × 90),(156 × 198),(373 × 326). These 9 priors are grouped into 3 different groups according to their scale. Each group is assigned to a specific feature map above in detecting objects.

In [8]:
anchors  = read_anchors('model_data/yolo_anchors.txt')
class_names = read_classes("model_data/coco_classes.txt")

print(anchors)
image_shape = (720., 1280.)    

[[ 10.  13.]
 [ 16.  30.]
 [ 33.  23.]
 [ 30.  61.]
 [ 62.  45.]
 [ 59. 119.]
 [116.  90.]
 [156. 198.]
 [373. 326.]]


### 3.2 - Loading a pretrained model

For the purpose of this project, I'm using a pretrained weights for Yolo V2.
You can download the weights <a href = "http://pjreddie.com/media/files/yolo.weights"> here</a>

These weights are generated with using pure numpy arrays, but luckily the authors of Yolo provided configuration files containing the full model information, which means we can convert the weights in any high level program language.
Allan Zelener wrote a function for weights conversion, which works well for Yolo V2, and I provided small modifications for Yolo V3 References are at the end of this notebook.

In [9]:
yolo_model = load_model("yolo.h5")


C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


This loads the weights of a trained YOLO model. Here's a summary of the layers your model contains.

In [10]:
yolo_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

**Note**: Before we go further, I’d like to point out that stride of the network, or a layer is defined as the ratio by which it downsamples the input. In the following examples, I will assume we have an input image of size 416 x 416.

YOLO v3 makes prediction at three scales, which are precisely given by downsampling the dimensions of the input image by 32, 16 and 8 respectively.

### 3.3 - Convert output of the model to usable bounding box tensors

yolo_outputs gave you all the predicted boxes of yolo_model in the correct format. You're now ready to perform filtering and select only the best boxes. Lets now call yolo_eval, which you had previously implemented, to do this.


In [11]:
from PIL import Image, ImageFont, ImageDraw
import os


image = Image.open('test/straight_lines1.jpg')
image_data = preprocess_image(image)

input_image_shape = K.placeholder(shape=(2, ))

score = 0.6
iou = 0.5

boxes, scores, classes = yolo_eval(yolo_model.output, anchors,
                len(class_names), input_image_shape,
                score_threshold=score, iou_threshold=0.45)

print(boxes) 
print(scores)
print(classes)

out_boxes, out_scores, out_classes = sess.run(
    [boxes, scores, classes],
    feed_dict={
        yolo_model.input: image_data,
        input_image_shape: [image.size[1], image.size[0]],
        K.learning_phase(): 0
    })



Tensor("concat_11:0", shape=(?, 4), dtype=float32)
Tensor("concat_12:0", shape=(?,), dtype=float32)
Tensor("concat_13:0", shape=(?,), dtype=int32)


You added `yolo_outputs` to your graph. This set of 4 tensors is ready to be used as input by your `yolo_eval` function.

### 3.5 - Run the graph on an image

Let the fun begin. You have created a (`sess`) graph that can be summarized as follows:

1. <font color='purple'> yolo_model.input </font> is given to `yolo_model`. The model is used to compute the output <font color='purple'> yolo_model.output </font>
2. <font color='purple'> yolo_model.output </font> is processed by `yolo_head`. It gives you <font color='purple'> yolo_outputs </font>
3. <font color='purple'> yolo_outputs </font> goes through a filtering function, `yolo_eval`. It outputs your predictions: <font color='purple'> scores, boxes, classes </font>

**Exercise**: Implement predict() which runs the graph to test YOLO on an image.
You will need to run a TensorFlow session, to have it compute `scores, boxes, classes`.

The code below also uses the following function:
```python
image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
```
which outputs:
- image: a python (PIL) representation of your image used for drawing boxes. You won't need to use it.
- image_data: a numpy-array representing the image. This will be the input to the CNN.

**Important note**: when a model uses BatchNorm (as is the case in YOLO), you will need to pass an additional placeholder in the feed_dict {K.learning_phase(): 0}.

In [12]:

def predict(sess, image):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the preditions.
    
    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """
    # Preprocess your image
    image = Image.fromarray(image.astype('uint8'), 'RGB')

    image_data = preprocess_image(image, model_image_size = (416, 416))

    out_boxes, out_scores, out_classes = sess.run(
        [boxes, scores, classes],
        feed_dict={
            yolo_model.input: image_data,
            input_image_shape: [image.size[1], image.size[0]],
            K.learning_phase(): 0
        })
    ### END CODE HERE ###

    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    drawer=draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
  

    return out_scores, out_boxes, out_classes, drawer

Run the following cell on the "test.jpg" image to verify that your function is correct.

In [13]:
def pipeline_final(img):

    #start = time.time()
    out_scores, out_boxes, out_classes, image = predict(sess, img)
    # = time.time()
    #print (end - start)
    return image




In [14]:
car_dict = {}

from moviepy.editor import VideoFileClip

# Reinitialize some global variables.
caches = [] 

output = 'custom_output.mp4'
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(pipeline_final) 
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video custom_output.mp4
[MoviePy] Writing video custom_output.mp4


100%|█████████▉| 1260/1261 [01:36<00:00, 13.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: custom_output.mp4 

Wall time: 1min 37s


**References**: 
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)
- Allan Zelener - [YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)
- The official YOLO website (https://pjreddie.com/darknet/yolo/) 
- Vivek Yadav [<a href="https://medium.com/@vivek.yadav/part-1-generating-anchor-boxes-for-yolo-like-network-for-vehicle-detection-using-kitti-dataset-b2fe033e5807">Generating Anchor boxes for Yolo-like network for vehicle detection using KITTI dataset]</a>
- Christopher Bourez's blog [<a href="http://christopher5106.github.io/object/detectors/2017/08/10/bounding-box-object-detectors-understanding-yolo.html">Bounding box object detectors: understanding YOLO, You Look Only Once</a>]
- Andrew NG Coursera : [<a href="https://www.coursera.org/learn/convolutional-neural-networks">Convolutional Neural Networks
</a>]
